In [ ]:
# if hit plotting library issues, try resetting LD path for julia
# can set in ~/.local/share/jupyter/kernels/
# @assert ENV["LD_LIBRARY_PATH"] == ""
import Pkg
pkgs = [
    "Revise",
]
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end
import Mycelia

In [ ]:
PROJECT_BASEDIR = dirname(pwd())
data_dir = joinpath(PROJECT_BASEDIR, "data")
genome_dir = mkpath(joinpath(data_dir, "genomes"))

In [ ]:
short_read_sets = unique(map(x -> match(r"^(.+\.\d+x)\.", x).captures[1], filter(x -> occursin(r"\.fna\.art", x) && occursin(r"\.fq\.gz", x) && !occursin("trimming_report", x) && !occursin("_val_", x), sort(readdir(genome_dir, join=true), by=x->filesize(x)))))

In [ ]:
threads = 16
memory = threads * 2
for short_read_set in short_read_sets
    # short_read_set = first(short_read_sets)
    forward = short_read_set * ".1_val_1.fq.gz"
    reverse = short_read_set * ".2_val_2.fq.gz"
    outdir = short_read_set * "_megahit"
    if !isdir(outdir)
        cmd = "$(Mycelia.MAMBA) run --live-stream -n megahit megahit --num-cpu-threads $(threads) -1 $(forward) -2 $(reverse) --out-dir $(outdir)"
        Mycelia.nersc_sbatch(
            job_name = "$(basename(outdir))",
            mail_user = "cameron.prybol@gmail.com",
            logdir = mkpath("$(homedir())/workspace/slurmlogs"),
            qos = "shared",
            cpus_per_task= threads,
            cmd = cmd)
    else
        @info "$(outdir) already present"
    end
end